# 이진 분류 모델 성능 평가: 다양한 지표와 시각화

이 노트북은 이진 분류(Binary Classification) 모델의 성능을 평가하는 데 사용되는 다양한 지표들을 심층적으로 다룹니다. 특히 **데이터 불균형** 상황에서 정확도(Accuracy)만으로는 모델의 성능을 제대로 평가하기 어렵기 때문에, 오차 행렬(Confusion Matrix), 정밀도(Precision), 재현율(Recall), F1-점수(F1-Score), ROC 곡선(ROC Curve) 및 AUC(Area Under the Curve)와 같은 지표들을 이해하고 활용하는 것이 중요합니다.

**사용 데이터셋**: `scikit-learn`의 유방암 진단 데이터셋 (양성/악성 분류)

### 1. 라이브러리 임포트 및 한글 폰트 설정

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # 시각화를 위한 라이브러리

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer # 유방암 데이터셋
from sklearn.preprocessing import StandardScaler # 데이터 스케일링

# 한글 폰트 설정 (Windows 기준)
plt.rcParams['font.family'] = 'Malgun Gothic' # 설치된 나눔고딕 폰트 이름으로 변경
# 마이너스 부호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

### 2. 데이터 로드 및 전처리

유방암 데이터셋을 불러와 모델 학습에 적합한 형태로 전처리합니다.

**레이블 변경**: 원본 데이터셋에서 `0`은 악성(malignant), `1`은 양성(benign)을 의미합니다. 일반적으로 '관심 있는 클래스' 또는 '탐지하고자 하는 클래스'를 양성(Positive, 1)으로 설정합니다. 여기서는 **악성 종양을 양성(1)**으로 간주하고 평가 지표를 계산하기 위해 레이블을 변경합니다.

In [ ]:
cancer = load_breast_cancer() 
X = pd.DataFrame(cancer.data, columns= cancer.feature_names)
y = cancer.target # 0:악성, 1:양성

# 악성(0)을 양성(1)으로, 양성(1)을 음성(0)으로 변경
y_relabelled = np.where(y == 0, 1, 0) 

print("원본 레이블 (처음 20개):", y[:20])
print("변경 후 레이블 (처음 20개):", y_relabelled[:20])

# 변경된 레이블의 클래스 분포 확인
unique_labels, counts = np.unique(y_relabelled, return_counts=True)
print("
변경 후 레이블 분포:", dict(zip(unique_labels, counts)))
# 0: 양성(benign), 1: 악성(malignant)으로 간주합니다.

#### 데이터 분할 및 스케일링

모델의 일반화 성능을 평가하기 위해 데이터를 훈련 세트와 테스트 세트로 분할하고, 특성 스케일링을 적용합니다.

**주의**: 스케일링은 **훈련 데이터에만 `fit`**하고, `fit`된 스케일러를 사용하여 훈련 데이터와 테스트 데이터 모두를 `transform`해야 합니다. 테스트 데이터에 `fit`을 다시 하면 데이터 누수(Data Leakage)가 발생하여 모델 성능이 과대평가될 수 있습니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_relabelled, random_state=1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) # 여기를 .transform()으로 수정했습니다.

print(f"훈련 데이터 형태: {X_train_scaled.shape}")
print(f"테스트 데이터 형태: {X_test_scaled.shape}")

### 3. 모델 학습 및 예측

로지스틱 회귀 모델을 사용하여 학습을 진행하고, 테스트 데이터에 대한 예측을 수행합니다.

In [ ]:
model = LogisticRegression(random_state=1, solver='liblinear')
model.fit(X_train_scaled, y_train)

# 예측된 클래스 (0 또는 1)
y_pred = model.predict(X_test_scaled)

# 양성 클래스(1, 악성)에 대한 예측 확률
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1] 

print("예측된 클래스 (처음 10개):", y_pred[:10])
print("예측 확률 (양성 클래스, 처음 10개):", np.round(y_pred_proba[:10], 4))

### 4. 모델 성능 평가

이제 다양한 평가 지표를 통해 모델의 성능을 자세히 분석해 보겠습니다.

#### 4.1. 오차 행렬 (Confusion Matrix)

모델의 예측이 실제 값과 얼마나 일치하는지 보여주는 표입니다. 이진 분류에서는 4가지 경우의 수를 가집니다.

- **TP (True Positive)**: 실제 양성(악성)을 양성(악성)으로 올바르게 예측
- **TN (True Negative)**: 실제 음성(양성)을 음성(양성)으로 올바르게 예측
- **FP (False Positive)**: 실제 음성(양성)을 양성(악성)으로 잘못 예측 (1종 오류)
- **FN (False Negative)**: 실제 양성(악성)을 음성(양성)으로 잘못 예측 (2종 오류, **가장 치명적인 오류인 경우가 많음**)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print("오차 행렬:")
print(cm)

# 오차 행렬 시각화
plt.figure(figsize=(7, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['양성(예측)', '악성(예측)'],
            yticklabels=['양성(실제)', '악성(실제)'])
plt.xlabel('예측 클래스')
plt.ylabel('실제 클래스')
plt.title('오차 행렬')
plt.show()

#### 4.2. 분류 리포트 (Classification Report)

정확도(Accuracy), 정밀도(Precision), 재현율(Recall), F1-점수(F1-Score)를 한눈에 볼 수 있는 요약 리포트입니다.

- **정확도 (Accuracy)**: `(TP + TN) / (전체 예측 수)`. 전체 예측 중 올바르게 예측한 비율. 데이터 불균형 시 오해의 소지가 있습니다.
- **정밀도 (Precision)**: `TP / (TP + FP)`. 모델이 '양성'이라고 예측한 것들 중 실제 '양성'인 것의 비율. (예: 스팸 메일함의 신뢰도)
- **재현율 (Recall / Sensitivity)**: `TP / (TP + FN)`. 실제 '양성'인 것들 중 모델이 '양성'으로 예측한 것의 비율. (예: 암 진단에서 실제 환자를 놓치지 않는 것)
- **F1-점수 (F1-Score)**: 정밀도와 재현율의 조화 평균. `2 * (Precision * Recall) / (Precision + Recall)`. 두 지표가 모두 중요할 때 사용되는 균형 잡힌 지표입니다.
- **Support**: 각 클래스의 실제 샘플 수.

In [ ]:
report = classification_report(y_test, y_pred, target_names=['양성(benign)', '악성(malignant)'])
print("분류 리포트:")
print(report)

#### 4.3. ROC 곡선 (Receiver Operating Characteristic Curve) 및 AUC (Area Under the Curve)

ROC 곡선은 분류 모델의 임계값(threshold)을 변경하면서 재현율(True Positive Rate, TPR)과 위양성률(False Positive Rate, FPR)의 변화를 시각화한 것입니다.

- **TPR (재현율)**: 실제 양성 중 올바르게 예측한 비율.
- **FPR**: 실제 음성 중 잘못 예측한 비율 (`FP / (FP + TN)`).

**AUC**는 ROC 곡선 아래의 면적을 의미하며, 0부터 1까지의 값을 가집니다. 1에 가까울수록 모델의 분류 성능이 좋다는 것을 의미하며, 0.5는 무작위 분류와 같습니다.

In [ ]:
# roc_curve 함수는 FPR, TPR, 임계값을 반환합니다.
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
auc_score = roc_auc_score(y_test, y_pred_proba)

plt.figure(figsize=(8, 7))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC 곡선 (AUC = {auc_score:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='기준선 (AUC = 0.5)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR) / Recall')
plt.title('ROC 곡선')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

### 5. 결론

이진 분류 모델의 성능을 평가할 때는 단순히 정확도 하나만 보는 것이 아니라, 오차 행렬, 정밀도, 재현율, F1-점수, ROC 곡선 및 AUC와 같은 다양한 지표들을 종합적으로 고려해야 합니다. 특히 문제의 특성(예: 오탐지/미탐지의 비용)에 따라 어떤 지표가 더 중요한지 판단하여 모델을 선택하고 최적화해야 합니다.